In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import numpy as np
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Set plotting style
sns.set(style="whitegrid")

In [3]:
# Load the datasets
try:
    trader_df = pd.read_csv("historical_data.csv")
    sentiment_df = pd.read_csv("fear_greed_index.csv")

    print("--- Trader Data (historical_data.csv) Head ---")
    print(trader_df.head())
    print("\nTrader Data Info:")
    trader_df.info()

    print("\n\n--- Sentiment Data (fear_greed_index.csv) Head ---")
    print(sentiment_df.head())
    print("\nSentiment Data Info:")
    sentiment_df.info()

except FileNotFoundError as e:
    print(f"Error loading file: {e}")
    print("Please make sure 'historical_data.csv' and 'fear_greed_index.csv' are uploaded.")

--- Trader Data (historical_data.csv) Head ---
                                      Account  Coin  Execution Price  \
0  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9769   
1  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9800   
2  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9855   
3  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9874   
4  0xae5eacaf9c6b9111fd53034a602c192a04e082ed  @107           7.9894   

   Size Tokens  Size USD Side     Timestamp IST  Start Position Direction  \
0       986.87   7872.16  BUY  02-12-2024 22:50        0.000000       Buy   
1        16.00    127.68  BUY  02-12-2024 22:50      986.524596       Buy   
2       144.09   1150.63  BUY  02-12-2024 22:50     1002.518996       Buy   
3       142.98   1142.04  BUY  02-12-2024 22:50     1146.558564       Buy   
4         8.73     69.75  BUY  02-12-2024 22:50     1289.488521       Buy   

   Closed PnL                                   Transacti

In [4]:
# 1. Process Sentiment Data
sentiment_df['date'] = pd.to_datetime(sentiment_df['date'])
sentiment_df.set_index('date', inplace=True)
sentiment_df.rename(columns={'value': 'sentiment_value'}, inplace=True)

# 2. Process Trader Data and create new features
trader_df['timestamp'] = pd.to_datetime(trader_df['Timestamp IST'], format='%d-%m-%Y %H:%M')
trader_df['date'] = pd.to_datetime(trader_df['timestamp'].dt.date)

# Create new features for advanced metrics
trader_df['is_win'] = trader_df['Closed PnL'] > 0
trader_df['gross_profit'] = trader_df['Closed PnL'].apply(lambda x: x if x > 0 else 0)
trader_df['gross_loss'] = trader_df['Closed PnL'].apply(lambda x: abs(x) if x < 0 else 0)

print("--- Processed Trader Data (with new feature columns) ---")
print(trader_df[['date', 'Closed PnL', 'is_win', 'gross_profit', 'gross_loss']].head())

--- Processed Trader Data (with new feature columns) ---
        date  Closed PnL  is_win  gross_profit  gross_loss
0 2024-12-02         0.0   False           0.0         0.0
1 2024-12-02         0.0   False           0.0         0.0
2 2024-12-02         0.0   False           0.0         0.0
3 2024-12-02         0.0   False           0.0         0.0
4 2024-12-02         0.0   False           0.0         0.0


In [5]:
# Group by date and aggregate to get daily stats
daily_trader_stats = trader_df.groupby('date').agg(
    total_volume_usd=('Size USD', 'sum'),
    total_pnl=('Closed PnL', 'sum'),
    trade_count=('Transaction Hash', 'count'),
    pnl_volatility=('Closed PnL', 'std'),
    daily_win_rate=('is_win', 'mean'),
    total_gross_profit=('gross_profit', 'sum'),
    total_gross_loss=('gross_loss', 'sum')
)

# --- Calculate Advanced Daily Metrics ---
daily_trader_stats['pnl_volatility'] = daily_trader_stats['pnl_volatility'].fillna(0)

# Profit Factor
daily_trader_stats['profit_factor'] = daily_trader_stats['total_gross_profit'] / daily_trader_stats['total_gross_loss']
daily_trader_stats['profit_factor'].replace([np.inf, -np.inf], np.nan, inplace=True)
daily_trader_stats['profit_factor'] = daily_trader_stats['profit_factor'].fillna(0)

# Avg PnL per Trade
daily_trader_stats['avg_pnl_per_trade'] = daily_trader_stats['total_pnl'] / daily_trader_stats['trade_count']
daily_trader_stats['avg_pnl_per_trade'] = daily_trader_stats['avg_pnl_per_trade'].fillna(0)

# Sharpe Ratio (Simple version: Return / Risk)
daily_trader_stats['sharpe_ratio'] = daily_trader_stats['avg_pnl_per_trade'] / daily_trader_stats['pnl_volatility']
daily_trader_stats['sharpe_ratio'].replace([np.inf, -np.inf], np.nan, inplace=True)
daily_trader_stats['sharpe_ratio'] = daily_trader_stats['sharpe_ratio'].fillna(0)

print("\n--- Daily Trader Metrics (with advanced metrics) ---")
print(daily_trader_stats.head())


--- Daily Trader Metrics (with advanced metrics) ---
            total_volume_usd   total_pnl  trade_count  pnl_volatility  \
date                                                                    
2023-05-01            477.00    0.000000            3        0.000000   
2023-12-05          50005.83    0.000000            9        0.000000   
2023-12-14         113203.35 -205.434737           11      179.663534   
2023-12-15          10609.95  -24.632034            2        4.474304   
2023-12-16          15348.77    0.000000            3        0.000000   

            daily_win_rate  total_gross_profit  total_gross_loss  \
date                                                               
2023-05-01        0.000000            0.000000          0.000000   
2023-12-05        0.000000            0.000000          0.000000   
2023-12-14        0.363636          321.798393        527.233130   
2023-12-15        0.000000            0.000000         24.632034   
2023-12-16        0.000000

/tmp/ipython-input-2561976379.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  daily_trader_stats['profit_factor'].replace([np.inf, -np.inf], np.nan, inplace=True)
/tmp/ipython-input-2561976379.py:26: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col

In [6]:
# Merge the daily trader stats with the sentiment data
merged_df = pd.merge(
    daily_trader_stats,
    sentiment_df,
    left_index=True,
    right_index=True,
    how='inner'
)

# Create the directory and save the enhanced daily data
os.makedirs('csv_files', exist_ok=True)
merged_df.to_csv('csv_files/merged_daily_data_enhanced.csv')

print("--- Merged Daily Data ---")
print(merged_df.head())
print("\nSuccessfully saved 'merged_daily_data_enhanced.csv' to 'csv_files/' directory.")

--- Merged Daily Data ---
            total_volume_usd   total_pnl  trade_count  pnl_volatility  \
date                                                                    
2023-05-01            477.00    0.000000            3        0.000000   
2023-12-05          50005.83    0.000000            9        0.000000   
2023-12-14         113203.35 -205.434737           11      179.663534   
2023-12-15          10609.95  -24.632034            2        4.474304   
2023-12-16          15348.77    0.000000            3        0.000000   

            daily_win_rate  total_gross_profit  total_gross_loss  \
date                                                               
2023-05-01        0.000000            0.000000          0.000000   
2023-12-05        0.000000            0.000000          0.000000   
2023-12-14        0.363636          321.798393        527.233130   
2023-12-15        0.000000            0.000000         24.632034   
2023-12-16        0.000000            0.000000        

In [7]:
# Group by sentiment classification to get the average of our metrics
sentiment_analysis = merged_df.groupby('classification')[[
    'total_volume_usd', 'total_pnl', 'trade_count', 'pnl_volatility',
    'daily_win_rate', 'profit_factor', 'sharpe_ratio'
]].mean()

print("\n--- Average Daily Metrics (Enhanced) by Sentiment ---")
print(sentiment_analysis)

# --- Visualization ---
os.makedirs('outputs', exist_ok=True)
sentiment_order = ['Extreme Fear', 'Fear', 'Neutral', 'Greed', 'Extreme Greed']

# List of metrics to plot
metrics_to_plot = {
    'total_pnl': 'Average Total Daily PnL (USD)',
    'total_volume_usd': 'Average Total Daily Volume (USD)',
    'trade_count': 'Average Daily Trade Count',
    'pnl_volatility': 'Average PnL Volatility (Risk)',
    'daily_win_rate': 'Average Daily Win Rate',
    'profit_factor': 'Average Profit Factor',
    'sharpe_ratio': 'Average Sharpe Ratio (Return/Risk)'
}

for metric, title in metrics_to_plot.items():
    plt.figure(figsize=(10, 6))
    sns.barplot(x=sentiment_analysis.index, y=metric, data=sentiment_analysis, order=sentiment_order)
    plt.title(f'{title} by Market Sentiment')
    plt.ylabel(title)
    plt.xlabel('Market Sentiment')
    plt.savefig(f'outputs/avg_{metric}_vs_sentiment.png')
    plt.close() # Close the plot to avoid displaying it in the notebook output

# Correlation Heatmap
plt.figure(figsize=(12, 8))
corr_cols = ['sentiment_value', 'total_volume_usd', 'total_pnl', 'trade_count', 'pnl_volatility', 'daily_win_rate', 'profit_factor', 'sharpe_ratio']
sns.heatmap(merged_df[corr_cols].corr(), annot=True, cmap='coolwarm', fmt='.2f')
plt.title('Correlation Heatmap of Metrics and Sentiment Value')
plt.tight_layout()
plt.savefig('outputs/correlation_heatmap_enhanced.png')
plt.close()

print("All plots saved to 'outputs/' directory.")


--- Average Daily Metrics (Enhanced) by Sentiment ---
                total_volume_usd     total_pnl  trade_count  pnl_volatility  \
classification                                                                
Extreme Fear        8.177447e+06  52793.589178  1528.571429      729.867572   
Extreme Greed       1.091800e+06  23817.292199   350.807018      278.656985   
Fear                5.311261e+06  36891.818040   679.527473      328.500183   
Greed               1.495246e+06  11140.566181   260.637306      247.820567   
Neutral             2.690180e+06  19297.323516   562.477612      292.949624   

                daily_win_rate  profit_factor  sharpe_ratio  
classification                                               
Extreme Fear          0.327341      52.300837     -0.039984  
Extreme Greed         0.467424   15845.016660      0.289535  
Fear                  0.329112   38748.535914      0.205461  
Greed                 0.335986    2989.795438      0.214639  
Neutral            

In [8]:
# Fit an Ordinary Least Squares model to test if 'total_pnl' is different across 'classification'
model = ols('total_pnl ~ C(classification)', data=merged_df).fit()

# Perform the ANOVA test
anova_table = sm.stats.anova_lm(model, typ=2)

print("\n--- ANOVA Test on Total PnL vs. Sentiment ---")
print(anova_table)


--- ANOVA Test on Total PnL vs. Sentiment ---
                         sum_sq     df        F    PR(>F)
C(classification)  5.691421e+10    4.0  2.79126  0.025911
Residual           2.416233e+12  474.0      NaN       NaN


In [9]:
# Merge sentiment data back onto the original trader_df for user-level analysis
trader_sentiment_df = pd.merge(
    trader_df,
    sentiment_df,
    left_on='date',
    right_index=True,
    how='left'
)
trader_sentiment_df.dropna(subset=['classification'], inplace=True)

# Group by Account and Sentiment Classification
user_sentiment_analysis = trader_sentiment_df.groupby(['Account', 'classification']).agg(
    total_pnl=('Closed PnL', 'sum'),
    total_trades=('Transaction Hash', 'count'),
    win_rate=('is_win', 'mean')
).unstack(fill_value=0) # Pivot the sentiment classifications into columns

# Save this user-level analysis to a CSV
user_sentiment_analysis.to_csv('csv_files/user_sentiment_analysis.csv')

print("\n--- User-Level PnL, Trade Count, & Win Rate by Sentiment (Sample) ---")
print(user_sentiment_analysis.head())
print("\nSaved 'user_sentiment_analysis.csv' to 'csv_files/'.")


--- User-Level PnL, Trade Count, & Win Rate by Sentiment (Sample) ---
                                                total_pnl                 \
classification                               Extreme Fear  Extreme Greed   
Account                                                                    
0x083384f897ee0f19899168e3b1bec365f52a9012  124769.221441  -40282.335904   
0x23e7a7f8d14b550961925fbfdaa92f5d195ba5bd   -4235.153051   35658.894935   
0x271b280974205ca63b716753467d5a371de622ab   16706.884912    1793.160215   
0x28736f43f1e871e6aa8b1148d38d4994275d72c4  -25666.012284  108009.960644   
0x2c229d22b100a7beb69122eed721cee9b24011dd   11415.247787   90895.050410   

                                                                         \
classification                                      Fear          Greed   
Account                                                                   
0x083384f897ee0f19899168e3b1bec365f52a9012  1.113374e+06  276719.270187   
0x23e7a7f8d14b550961